<a href="https://colab.research.google.com/github/aghabayli/ensemble/blob/master/Hard_ensembling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Connect to GPU
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
#Connect to the google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
#Install dependencies

#!apt-get install python3-venv
#!python3 -m venv myenv/env_ensemble
#!source myenv/env_ensemble/bin/activate
!pip uninstall numpy==1.16.3
!pip install numpy==1.16.1
#!pip uninstall kashgari
!pip install /content/gdrive/My\ Drive/Kashgari/

# Data

In [2]:
#Import libraries
from keras.datasets import reuters
import numpy as np
from sklearn.model_selection import train_test_split

#Download the Reuters data set from Keras
(X_data, y_data), (X_test, y_test)  = reuters.load_data(path="../reuters.npz", num_words=None,skip_top=0,maxlen=None,test_split=0.2,seed=113,start_char=1,oov_char=2,index_from=3)

#Split validation data
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.60, random_state=113)

Using TensorFlow backend.


In [3]:
#Print example of input data
print(X_train[0])
print(y_train[0])


[1, 37, 412, 961, 202, 10, 146, 1315, 878, 466, 750, 445, 7, 91, 6, 10, 135, 32, 767, 11, 58, 83, 59, 20, 5, 4, 2023, 4, 1427, 213, 8, 7, 93, 878, 466, 961, 202, 27, 42, 8207, 553, 360, 639, 6, 12, 1477, 11, 58, 83, 72, 20, 16, 8, 4, 1832, 4203, 135, 7, 91, 348, 5739, 10745, 11480, 202, 10, 146, 2442, 18, 6, 32, 818, 11, 58, 83, 63, 20, 22, 6906, 12, 160, 11, 160, 32, 20, 91, 34, 4, 8454, 4494, 433, 22, 10, 2580, 54, 1489, 5, 160, 59, 20, 4, 1427, 213, 8, 961, 40, 1909, 6, 136, 109, 11, 13, 4, 99, 216, 7, 117, 109, 206, 4, 91, 302, 5, 750, 18, 34, 4, 473, 499, 224, 4045, 62, 99, 2344, 7, 10950, 1427, 262, 7221, 11801, 8, 137, 2554, 57, 994, 488, 305, 2260, 7, 295, 277, 961, 281, 4, 585, 29, 36, 152, 4, 54, 216, 4, 466, 135, 5, 4046, 34, 420, 109, 11, 34, 429, 1049, 184, 4, 1832, 135, 34, 54, 420, 52, 320, 7, 252, 1600, 17, 12]
30


In [0]:
#Constructing data to available input format at Kashgari
X_train = [[str(j) for j in i ]for i in X_train ]
y_train = [ str(i) for i in y_train ]

X_val = [[str(j) for j in i ]for i in X_val ]
y_val = [ str(i) for i in y_val ]

X_test = [[str(j) for j in i ]for i in X_test ]
y_test = [ str(i) for i in y_test ]

In [5]:
#Print example after constructing
print(X_train[0])
y_train[0]

['1', '37', '412', '961', '202', '10', '146', '1315', '878', '466', '750', '445', '7', '91', '6', '10', '135', '32', '767', '11', '58', '83', '59', '20', '5', '4', '2023', '4', '1427', '213', '8', '7', '93', '878', '466', '961', '202', '27', '42', '8207', '553', '360', '639', '6', '12', '1477', '11', '58', '83', '72', '20', '16', '8', '4', '1832', '4203', '135', '7', '91', '348', '5739', '10745', '11480', '202', '10', '146', '2442', '18', '6', '32', '818', '11', '58', '83', '63', '20', '22', '6906', '12', '160', '11', '160', '32', '20', '91', '34', '4', '8454', '4494', '433', '22', '10', '2580', '54', '1489', '5', '160', '59', '20', '4', '1427', '213', '8', '961', '40', '1909', '6', '136', '109', '11', '13', '4', '99', '216', '7', '117', '109', '206', '4', '91', '302', '5', '750', '18', '34', '4', '473', '499', '224', '4045', '62', '99', '2344', '7', '10950', '1427', '262', '7221', '11801', '8', '137', '2554', '57', '994', '488', '305', '2260', '7', '295', '277', '961', '281', '4', '58

'30'

# Training

In [0]:
#Import libraries to train
from kashgari.tasks.classification import CNNModel
from sklearn.metrics import accuracy_score, confusion_matrix
from numpy import dstack

In [0]:
def prediction(models, X_data, y_data = None):

  predictions = list()

  #Predict for each individual model
  for model in models:
    #Soft classification prediction
    pr = model.predict(X_data)
    
    #model.evaluate(X_data, y_data)
    
    #Stacking
    predictions.append(pr)
    
  #Transform to feed as a input  
  predictions =  np.array(predictions).T
    
  return predictions

In [11]:
predictions = list()
#Create a list to pre-ensem ble models
models = list()

#Number of indivsual models
n=5

#Train n individual models
for i in range(n):
    # CNN Model
    classifier = CNNModel()
    #Fit train data
    classifier.fit(X_train,y_train, multi_label=True, epochs = 3)
    #Append to list trained classifier
    models.append(classifier)
    #Predict soft classification on train data
    predicted = classifier.predict(X_train)
    #Stack all predictions 
    predictions.append(predicted)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 450)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 450, 100)          959800    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 446, 128)          64128     
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_12 (Dense)             (None, 47)                3055      
Total params: 1,035,239
Trainable params: 1,035,239
Non-trainable params: 0
_________________________________________________________________


### Predicting from indivdual models and training ensemble 

In [24]:
#CNN model from Kashgari
classifier = CNNModel()

#Predicting from pre-ensemble models   
val_predictions = prediction(models, X_val)

#Structure predictions to Kashgari input format
X_predict = [[str(j) for j in i ]for i in val_predictions ]

#Fit predicted probabilities as the input to the model
classifier.fit(X_predict, y_val, epochs=5)

#Predict on final model
ensemble_pr = classifier.predict(X_predict)

#print(confusion_matrix(y_val, ensemble_pr))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 5)                 0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 5, 100)            1100      
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 1, 128)            64128     
_________________________________________________________________
global_max_pooling1d_11 (Glo (None, 128)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_22 (Dense)             (None, 47)                3055      
Total params: 76,539
Trainable params: 76,539
Non-trainable params: 0
_________________________________________________________________
Epoch 

### Classification of test data

In [25]:
#Predicting from pre-ensemble models
test_predictions = prediction(models, X_test)
    
#Predict on final model
X_predict = [[str(j) for j in i ]for i in test_predictions]

#Final accuracy
print("Ensemble accuracy:")
print(accuracy_score(classifier.predict(X_predict), y_test))

Ensemble accuracy:
0.6869991095280499


# Treating Classification Outputs as Additional Features

In [28]:
#CNN model from Kashgari
classifier = CNNModel()

#Predicting from pre-ensemble models   
val_predictions = prediction(models, X_val)

#Concatinating probabilities from indivudal models with input data
X_predict = [[str(j) for j in i ] + k for k,i in zip(X_val, val_predictions)]


#Fit predicted probabilities as the input to the model
classifier.fit(X_predict, y_val, epochs=3)

#Predict on final model
ensemble_pr = classifier.predict(X_predict)


#classifier.evaluate(X_predict, y_val)
#print(confusion_matrix(y_val, ensemble_pr))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 466)               0         
_________________________________________________________________
embedding_13 (Embedding)     (None, 466, 100)          1148700   
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 462, 128)          64128     
_________________________________________________________________
global_max_pooling1d_13 (Glo (None, 128)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_26 (Dense)             (None, 47)                3055      
Total params: 1,224,139
Trainable params: 1,224,139
Non-trainable params: 0
_________________________________________________________________


In [29]:
#Predicting from pre-ensemble models
test_predictions = prediction(models, X_test)
   

#Predict on final model
X_predict = [[str(j) for j in i ] + k for k,i in zip(X_test, test_predictions) ]

#Final accuracy
print("Ensemble accuracy:")
print(accuracy_score(classifier.predict(X_predict), y_test))

Ensemble accuracy:
0.684772929652716
